# 1. Import Libraries



In [110]:
import re
import os
import random

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy

from collections import defaultdict
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS, CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

import nltk
nltk.download('stopwords');
nltk.download('wordnet');

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.porter import *
from nltk.stem.wordnet import WordNetLemmatizer


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


# 2. Load Data


In [111]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [112]:
data = df = pd.read_csv('/content/drive/MyDrive/Applied_ML/data/10_core_Video_Games_Processed.csv')
data.head()

,overall,verified,reviewTime,reviewerID,asin,reviewerName,reviewText,summary,category,description,title,also_buy,brand,feature,also_view,similar_item,details
0,5.0,False,2001-01-01,A3BEJOL2X8PS8I,B00004TC6E,ATHF Fan,Before I start this review let me get somethin...,The best RPG... Ever!,"['Video Games', 'Retro Gaming & Microconsoles'...","[""While it's officially a sequel to the immens...",Chrono Cross - PlayStation,"['B00002R28C', 'B00005LOXE', 'B00004Y57G', 'B0...",by\n \n Square Enix,['PSX'],"['B0045L3SNQ', 'B00005LOXE', 'B00004Y57G', 'B0...",NaN,NaN
1,5.0,False,2003-01-01,A3E0MSKBIWMU6F,B000069D39,obryanstars,"Well, at first, getting used to the controls i...","A Great Game, a HALO for the PS2 (maybe)","['Video Games', 'Retro Gaming &amp; Microconso...",['The evil race of TimeSplitters is traveling ...,Time Splitters 2 - PlayStation 2,"['B00004WKN9', 'B0006ZJP8M', 'B0009O7HVM', 'B0...",by\n \n Square Enix,"[""This time you've got to stop the evil lords ...","['B0006ZJP8M', 'B00006AVB1', 'B00004WKN9', 'B0...",NaN,NaN
2,4.0,False,2003-01-01,A3KI73MFVL0QWH,B00005NN5J,GAMEMASTER,this game is pretty good its graphics and game...,cool game,"['Video Games', 'Retro Gaming & Microconsoles'...","[""Danger takes wing again, as Namco's breaknec...",Namco Ace Combat 4 Shattered Skies,"['B01L1P49OA', 'B00JA81WQE', 'B0000C7GHG', 'B0...",Playstation,"['Soar through the blue in 24 missions, piloti...","['B00065QTT0', 'B01A0P8NBC', 'B015PJ68BK', 'B0...",NaN,NaN
3,5.0,False,2003-01-01,A3E0MSKBIWMU6F,B00007E1OC,obryanstars,The Lord of the Rings: The Two Towers (the gam...,"So Much Fun, So Many Things To Do! A Classic!!!","['Video Games', 'Retro Gaming & Microconsoles'...","['Features detailed tactical component.', '<I>...",Lord of the Rings The Two Towers - Gamecube,"['B00009VE6B', 'B0002I9RQI', 'B01LWA474D', 'B0...",by\n \n Electronic Arts,"[""The evil Wizard Saruman has built is buildin...","['B00009VE6B', 'B00008KTW7', 'B0002I9RQI', 'B0...",NaN,NaN
4,4.0,False,2003-01-01,A3713PP4MADKED,B00006GSNZ,Nick Watkins,"I got this game for Christmas, and played it a...",Good...BUT TOO HARD!!!!!!,"['Video Games', 'Retro Gaming & Microconsoles'...","[""Battle the evil forces of Mordor and experie...",Lord of the Rings The Two Towers - PlayStation 2,"['B00009VE6D', 'B00006LEM9', 'B0002IQD1U', 'B0...",by\n \n Electronic Arts,['Enter the pastoral world of Middle Earth whe...,"['B00009VE6D', 'B00006LEM9', 'B0002IQD1U', 'B0...",NaN,NaN


# 3. Preprocessing

In [113]:
data[['description','reviewText', 'summary', 'also_buy', 'also_view']] = data[['description','reviewText', 'summary', 'also_buy', 'also_view']].fillna('')

meta = data[['asin', 'title', 'also_buy', 'also_view', 'overall', 'description', 'reviewText', 'summary']]
meta = meta.groupby(['asin', 'title', 'also_buy', 'also_view'], as_index=False).agg({'overall':'mean', 
                                                                                     'description':'first',
                                                                                     'reviewText': lambda x: ''.join(str(x.values)),
                                                                                     'summary': lambda x: ''.join(str(x.values))})

meta.head()

,asin,title,also_buy,also_view,overall,description,reviewText,summary
0,0804161380,Legend of Zelda Box Set Prima Official Game Guide,"['1911015214', '1911015222', '1506703356', '11...","['1911015486', '1506703356', '1616550414', '15...",4.733333,"[""It's 6 Legend Of Zelda strategy guides, offe...",['amazing\n\ncool awesome interesting guides o...,['Great' 'Five Stars' 'Five Stars' 'Epic colle...
1,9629971372,Dotop Sony Playstation 2 PS2 64MB Memory Card,"['B01LY4QRT8', 'B0000C7GHG', 'B00004SQPD', 'B0...",,4.588235,"[""This is Memory Card 128MB for PlayStation 2 ...",['How can you go wrong with obtaining a 128 Me...,['Works on 95% of games!' 'nice' 'As Expected'...
2,B000006OVJ,Mega Man Legends,"['B00004XONL', 'B0045L3SNQ', 'B0009XBWY6', 'B0...","['B0045L3SNQ', 'B00004XONL', 'B00000K1VC', 'B0...",4.400000,['It is a world covered by endless water. Only...,['Well as far as the person who sold this to m...,['Hmm' 'Mega Man Legends For The PlayStation'\...
3,B000006P0K,Tekken 2,"['B00000K2X5', 'B000006P0J', 'B0000664JF', 'B0...","['B00000K2X5', 'B0002CHJ8C', 'B0000664JF', 'B0...",5.000000,"['New green label', ""They're all here Heihachi...","[""Same goes for Tekken 2: played but never own...",['Tekken 2' 'Five Stars' 'One of the best figh...
4,B000006RGR,Tomb Raider II,"['B00000DMB4', 'B008LYNLFI', 'B00001ZT9E', 'B0...","['B008LYNLFI', 'B00000DMB4', 'B00002SFNG', 'B0...",4.833333,"[""Lara's Back. Explore new worlds. Awesome Cla...","[""Tomb Raider II is the best of the series, fo...",['best tomb raider' 'A better game than the or...


In [114]:
process = lambda x: str(x).replace(' ', '').replace("'", '')[1:-1].split(',')
cleanText = lambda x: (re.sub(r'[^A-Za-z0-9 ]+', '', x)).lower()

meta[['description', 'reviewText', 'summary', 'also_buy', 'also_view']] = meta.apply({'description': lambda x: cleanText(x), 
                                                                                      'reviewText': lambda x: cleanText(x), 
                                                                                      'summary': lambda x: cleanText(x), 
                                                                                      'also_buy': lambda x: process(x), 
                                                                                      'also_view': lambda x: process(x)})

meta.head()

,asin,title,also_buy,also_view,overall,description,reviewText,summary
0,0804161380,Legend of Zelda Box Set Prima Official Game Guide,"[1911015214, 1911015222, 1506703356, 110189843...","[1911015486, 1506703356, 1616550414, 150670740...",4.733333,its 6 legend of zelda strategy guides offered ...,amazingnncool awesome interesting guides of th...,great five stars five stars epic collection fi...
1,9629971372,Dotop Sony Playstation 2 PS2 64MB Memory Card,"[B01LY4QRT8, B0000C7GHG, B00004SQPD, B000SEU92...",[],4.588235,this is memory card 128mb for playstation 2 ps...,how can you go wrong with obtaining a 128 mega...,works on 95 of games nice as expected my first...
2,B000006OVJ,Mega Man Legends,"[B00004XONL, B0045L3SNQ, B0009XBWY6, B00005MA9...","[B0045L3SNQ, B00004XONL, B00000K1VC, B000050FA...",4.400000,it is a world covered by endless water only sm...,well as far as the person who sold this to me ...,hmm mega man legends for the playstation megam...
3,B000006P0K,Tekken 2,"[B00000K2X5, B000006P0J, B0000664JF, B0002CHJ8...","[B00000K2X5, B0002CHJ8C, B0000664JF, B000006P0...",5.000000,new green label theyre all here heihachi paul ...,same goes for tekken 2 played but never owned ...,tekken 2 five stars one of the best fighting g...
4,B000006RGR,Tomb Raider II,"[B00000DMB4, B008LYNLFI, B00001ZT9E, B000006RG...","[B008LYNLFI, B00000DMB4, B00002SFNG, B00001ZT9...",4.833333,laras back explore new worlds awesome classic ...,tomb raider ii is the best of the series follo...,best tomb raider a better game than the origin...


In [115]:
stop = stopwords.words('english')
removeStopWords = lambda x:' '.join([word for word in x.split() if word not in (stop)])

meta[['description', 'reviewText', 'summary']] = meta.apply({
    'description': lambda x: removeStopWords(x), 
    'reviewText': lambda x: removeStopWords(x), 
    'summary': lambda x: removeStopWords(x)})

meta.head()

,asin,title,also_buy,also_view,overall,description,reviewText,summary
0,0804161380,Legend of Zelda Box Set Prima Official Game Guide,"[1911015214, 1911015222, 1506703356, 110189843...","[1911015486, 1506703356, 1616550414, 150670740...",4.733333,6 legend zelda strategy guides offered numbere...,amazingnncool awesome interesting guides legen...,great five stars five stars epic collection fi...
1,9629971372,Dotop Sony Playstation 2 PS2 64MB Memory Card,"[B01LY4QRT8, B0000C7GHG, B00004SQPD, B000SEU92...",[],4.588235,memory card 128mb playstation 2 ps2 building b...,go wrong obtaining 128 megabyte memory card ps...,works 95 games nice expected first ps2 memory ...
2,B000006OVJ,Mega Man Legends,"[B00004XONL, B0045L3SNQ, B0009XBWY6, B00005MA9...","[B0045L3SNQ, B00004XONL, B00000K1VC, B000050FA...",4.400000,world covered endless water small patches land...,well far person sold goes game came described ...,hmm mega man legends playstation megaman ocari...
3,B000006P0K,Tekken 2,"[B00000K2X5, B000006P0J, B0000664JF, B0002CHJ8...","[B00000K2X5, B0002CHJ8C, B0000664JF, B000006P0...",5.000000,new green label theyre heihachi paul phoenix k...,goes tekken 2 played never owned im officially...,tekken 2 five stars one best fighting games ti...
4,B000006RGR,Tomb Raider II,"[B00000DMB4, B008LYNLFI, B00001ZT9E, B000006RG...","[B008LYNLFI, B00000DMB4, B00002SFNG, B00001ZT9...",4.833333,laras back explore new worlds awesome classic ...,tomb raider ii best series followed original s...,best tomb raider better game original perfect ...


In [116]:
lemmatizer = WordNetLemmatizer()
lemmtizeText = lambda sentence: ' '.join([lemmatizer.lemmatize(w) for w in nltk.word_tokenize(sentence)])

meta[['description', 'reviewText', 'summary']] = meta.apply({
    'description': lambda x: lemmtizeText(x), 
    'reviewText': lambda x: lemmtizeText(x), 
    'summary': lambda x: lemmtizeText(x)})

meta.head()

,asin,title,also_buy,also_view,overall,description,reviewText,summary
0,0804161380,Legend of Zelda Box Set Prima Official Game Guide,"[1911015214, 1911015222, 1506703356, 110189843...","[1911015486, 1506703356, 1616550414, 150670740...",4.733333,6 legend zelda strategy guide offered numbered...,amazingnncool awesome interesting guide legend...,great five star five star epic collection five...
1,9629971372,Dotop Sony Playstation 2 PS2 64MB Memory Card,"[B01LY4QRT8, B0000C7GHG, B00004SQPD, B000SEU92...",[],4.588235,memory card 128mb playstation 2 ps2 building b...,go wrong obtaining 128 megabyte memory card ps...,work 95 game nice expected first ps2 memory ca...
2,B000006OVJ,Mega Man Legends,"[B00004XONL, B0045L3SNQ, B0009XBWY6, B00005MA9...","[B0045L3SNQ, B00004XONL, B00000K1VC, B000050FA...",4.400000,world covered endless water small patch land s...,well far person sold go game came described we...,hmm mega man legend playstation megaman ocarin...
3,B000006P0K,Tekken 2,"[B00000K2X5, B000006P0J, B0000664JF, B0002CHJ8...","[B00000K2X5, B0002CHJ8C, B0000664JF, B000006P0...",5.000000,new green label theyre heihachi paul phoenix k...,go tekken 2 played never owned im officially b...,tekken 2 five star one best fighting game time...
4,B000006RGR,Tomb Raider II,"[B00000DMB4, B008LYNLFI, B00001ZT9E, B000006RG...","[B008LYNLFI, B00000DMB4, B00002SFNG, B00001ZT9...",4.833333,laras back explore new world awesome classic g...,tomb raider ii best series followed original s...,best tomb raider better game original perfect ...


In [117]:
meta['without reviewText'] = meta['summary'] + meta['description']
meta['with reviewText'] = meta['summary'] + meta['description'] + meta['reviewText']
meta.head()

,asin,title,also_buy,also_view,overall,description,reviewText,summary,without reviewText,with reviewText
0,0804161380,Legend of Zelda Box Set Prima Official Game Guide,"[1911015214, 1911015222, 1506703356, 110189843...","[1911015486, 1506703356, 1616550414, 150670740...",4.733333,6 legend zelda strategy guide offered numbered...,amazingnncool awesome interesting guide legend...,great five star five star epic collection five...,great five star five star epic collection five...,great five star five star epic collection five...
1,9629971372,Dotop Sony Playstation 2 PS2 64MB Memory Card,"[B01LY4QRT8, B0000C7GHG, B00004SQPD, B000SEU92...",[],4.588235,memory card 128mb playstation 2 ps2 building b...,go wrong obtaining 128 megabyte memory card ps...,work 95 game nice expected first ps2 memory ca...,work 95 game nice expected first ps2 memory ca...,work 95 game nice expected first ps2 memory ca...
2,B000006OVJ,Mega Man Legends,"[B00004XONL, B0045L3SNQ, B0009XBWY6, B00005MA9...","[B0045L3SNQ, B00004XONL, B00000K1VC, B000050FA...",4.400000,world covered endless water small patch land s...,well far person sold go game came described we...,hmm mega man legend playstation megaman ocarin...,hmm mega man legend playstation megaman ocarin...,hmm mega man legend playstation megaman ocarin...
3,B000006P0K,Tekken 2,"[B00000K2X5, B000006P0J, B0000664JF, B0002CHJ8...","[B00000K2X5, B0002CHJ8C, B0000664JF, B000006P0...",5.000000,new green label theyre heihachi paul phoenix k...,go tekken 2 played never owned im officially b...,tekken 2 five star one best fighting game time...,tekken 2 five star one best fighting game time...,tekken 2 five star one best fighting game time...
4,B000006RGR,Tomb Raider II,"[B00000DMB4, B008LYNLFI, B00001ZT9E, B000006RG...","[B008LYNLFI, B00000DMB4, B00002SFNG, B00001ZT9...",4.833333,laras back explore new world awesome classic g...,tomb raider ii best series followed original s...,best tomb raider better game original perfect ...,best tomb raider better game original perfect ...,best tomb raider better game original perfect ...


# 4. Model Building

In [143]:
def tf_similarity(df, col):
  tf = TfidfVectorizer()
  mat = tf.fit_transform(df[col])
  # print('Tfidf Matrix Shape: {}'.format(mat.shape))
  sim = linear_kernel(mat, mat)
  # print('Similarity Matrix Shape: {}\n'.format(sim.shape))
  return sim

def cv_similarity(df, col):
  cv = CountVectorizer()
  mat = cv.fit_transform(df[col])
  # print('CV Matrix Shape: {}'.format(mat.shape))
  sim = linear_kernel(mat, mat)
  # print('Similarity Matrix Shape: {}\n'.format(sim.shape))
  return sim

def get_recommendations(df, col, method, id):
  # similarity matrix
  sim = tf_similarity(df, col) if method == 'tfidf' else cv_similarity(df, col)

  # find recommendations
  ids = df['asin']
  indices = pd.Series(df.index, index=df['asin'])
  idx = indices[id]
  sim_scores = list(enumerate(sim[idx]))
  sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
  sim_scores = sim_scores[1:31]
  prod_indices = [i[0] for i in sim_scores]
  return ids.iloc[prod_indices]

def display_recommendations(df, col, method, id):
  for rid in get_recommendations(df, col, method, id).head(10).values:
    print(df[df['asin'] == rid]['title'].unique())

In [119]:
without_reviewText_tf = tf_similarity(meta, 'without reviewText')

Tfidf Matrix Shape: (3784, 57232)
Similarity Matrix Shape: (3784, 3784)



In [120]:
without_reviewText_tf_sim.shape

(3784, 3784)

In [121]:
display_recommendations(meta, 'without reviewText', 'tfidf', '0804161380')

Tfidf Matrix Shape: (3784, 57232)
Similarity Matrix Shape: (3784, 3784)

['The Legend of Zelda:  Phantom Hourglass']
['The Legend of Zelda: The Wind Waker']
['The Legend of Zelda: A Link Between Worlds 3D']
['The Legend of Zelda: Twilight Princess HD - Wii U']
['The Legend of Zelda: Skyward Sword']
['The Legend of Zelda: Twilight Princess']
['Zelda II: The Adventure of Link']
['The Legend of Zelda: Spirit Tracks']
['The Legend of Zelda']
['The Legend of Zelda: TriForce Heroes - 3DS']


In [122]:
display_recommendations(meta, 'with reviewText', 'cv', '0804161380')

CV Matrix Shape: (3784, 329436)
Similarity Matrix Shape: (3784, 3784)

['Bloodborne']
["Assassin's Creed"]
['Uncharted 2: Among Thieves - Playstation 3']
['Metal Gear Solid 4: Guns of the Patriots']
['Destiny - Standard Edition - PlayStation 4']
[' Mass Effect 2 Platinum Hits']
['Wii']
['Final Fantasy X']
['Diablo III']
['Grand Theft Auto IV']


# 5. Evaluation

In [176]:
def evaluate(items, df, col, method):
  arr_buy = []
  arr_view = []
  print('--- Building Recommendations ---')
  i = 1
  for id in items:
    recommendations_ = get_recommendations(df, col, method, id)[0:5]
    recommendations_ = set(recommendations_)

    also_buy_ = list(df[df['asin']==id].also_buy)[0]
    also_buy_ = set(also_buy_)

    also_view_ = list(df[df['asin']==id].also_view)[0]
    also_view_ = set(also_view_)

    intr_buy = also_buy_.intersection(recommendations_)
    intr_view = also_view_.intersection(recommendations_)
    
    arr_buy.append(len(intr_buy)/len(recommendations_))
    arr_view.append(len(intr_view)/len(recommendations_))

    if (i%20 == 0):
      print('{} IDs processed'.format(i))
    i += 1

  print('\n\n--- Evaluation Result ---')
  buy_score = sum(arr_buy)/len(arr_buy)
  view_score = sum(arr_view)/len(arr_view)

  print('Also Buy Score: {}'.format(buy_score))
  print('Also View Score: {}'.format(view_score))
  return buy_score, view_score

In [177]:
random.seed(1024)
rdf = meta.sample(100)
tfidf_without_reviewText_buy_score = evaluate(list(rdf.asin.values), meta, 'without reviewText', 'tfidf')

--- Building Recommendations ---
20 IDs processed
40 IDs processed
60 IDs processed
80 IDs processed
100 IDs processed


--- Evaluation Result ---
Also Buy Score: 0.2499999999999999
Also View Score: 0.3179999999999998
